This project aims to recommend products to the user when he/she already has added one to cart. 

Apriori algorithm will be used. It is an algorithm to detect frequent item sets. It works by first identifying individual items that satisfy a minimum occurrence threshold. It then extends the item set, by looking at all possible pairs that still satisfy the specified threshold. As a final step, we calculate the following three metrics that are central to this algorithm.

support: This is the percentage of orders that contains the item or item set. For an individual item A, if it is seen in 3 of 5 orders:
 
support(A) = 3/5 or 60%

If there are 2 items A and B, support value is the percentage that A and B occur together:

support(A, B) = Freq(A,B) / N where Freq(A,B) is the number that A and B are together and N is the overall order amount. 

confidence: Confidence is the percentage that for an A item already purchased, B item was also purchased with A. 

confidence(A->B) = Freq(A,B) / Freq(A) where Freq(A,B) is the number that A and B are purchased together and Freq(A) is the number where A is purchased.
 
lift: Lift value indicates that for the A item purchased, how many times the percentage of B purchasing increased. 

lift = support(A,B) / (support(A) * support(B))

lift > 1 implies that there is a positive relationship between A and B.

The dataset contains the purchasing data of a UK based online souvenir store between 12/01/2009 and 12/09/2011.

To narrow down, the user will be chosen among Germany customers that shopped between 2010 and 2011.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-retail-ii-data-set-from-ml-repository/Year 2010-2011.csv
/kaggle/input/online-retail-ii-data-set-from-ml-repository/Year 2009-2010.csv
/kaggle/input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx


# 1. Data Preprocessing

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 287 kB/s            


In [4]:
df_ = pd.read_excel('../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx', sheet_name="Year 2010-2011")
df = df_.copy()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# The function is used to state the thresholds for the outlier values.

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


In [6]:
# The outlier values are replaced with the threshold values.

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [7]:
# The NaN values are removed. In the Invoice column, the invoices starting with C shows the cancelled orders. Since the cancelled orders take place as 
# negative values in the Quantity and Price columns, the negative values are eliminated from these columns.   

def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe


df = retail_data_prep(df)

# 2. Preparing the data for ARL

In [8]:
# The customer will be chosen from Germany. So a new dataset where only customers in Germany are involved is composed. 

df_ger = df[df["Country"] == "Germany"]

df_ger.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6.0,2010-12-01 13:04:00,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6.0,2010-12-01 13:04:00,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12.0,2010-12-01 13:04:00,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12.0,2010-12-01 13:04:00,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12.0,2010-12-01 13:04:00,1.95,12662.0,Germany


#

In [9]:
# The table the data chosen from should look like the table below. If a product is in the invoice, there should be write 1, otherwise 0. 

# Description   NINE DRAWER OFFICE TIDY   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET...
# Invoice
# 536370                              0                                 1                       0..
# 536852                              1                                 0                       1..
# 536974                              0                                 0                       0..
# 537065                              1                                 0                       0..
# 537463                              0                                 0                       1..
 


In [10]:
# The number of products ordered at each order:

df_ger.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).head(20)


Quantity
Invoice Description                                  
536527  3 HOOK HANGER MAGIC GARDEN               12.0
        5 HOOK HANGER MAGIC TOADSTOOL            12.0
        5 HOOK HANGER RED MAGIC TOADSTOOL        12.0
        ASSORTED COLOUR LIZARD SUCTION HOOK      24.0
        CHILDREN'S CIRCUS PARADE MUG             12.0
        HOMEMADE JAM SCENTED CANDLES             12.0
        HOT WATER BOTTLE BABUSHKA                 4.0
        JUMBO BAG OWLS                           10.0
        JUMBO BAG WOODLAND ANIMALS               10.0
        MULTI COLOUR SILVER T-LIGHT HOLDER       12.0
        PACK 3 FIRE ENGINE/CAR PATCHES           12.0
        PICTURE DOMINOES                         12.0
        POSTAGE                                   1.0
        ROTATING SILVER ANGELS T-LIGHT HLDR       6.0
        SET OF 6 T-LIGHTS SANTA                   6.0
536840  6 RIBBONS RUSTIC CHARM                   12.0
        60 CAKE CASES VINTAGE CHRISTMAS          24.0
        60 TEATIME FAIRY CAKE CASES              24.0
        CAKE STAND WHITE TWO TIER LACE            2.0
        JAM JAR WITH GREEN LID                   12.0

In [11]:
# For having one invoice per row, unstack method is used. If a product is in the invoice, its amount will be on the table. Otherwise it will be NaN.
# Only 5 rows and 5 columns are shown with iloc method.

df_ger.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().iloc[0:5, 0:5]


Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  RED SPOT GIFT BAG LARGE  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536527                                 NaN                NaN                          NaN                      NaN                              NaN
536840                                 NaN                NaN                          NaN                      NaN                              NaN
536861                                 NaN                NaN                          NaN                      NaN                              NaN
536967                                 NaN                NaN                          NaN                      NaN                              NaN
536983                                 NaN                NaN                          NaN                      NaN                              NaN

In [12]:
# The NaN values are replaced with 0.

df_ger.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().fillna(0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  RED SPOT GIFT BAG LARGE  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536527                                 0.0                0.0                          0.0                      0.0                              0.0
536840                                 0.0                0.0                          0.0                      0.0                              0.0
536861                                 0.0                0.0                          0.0                      0.0                              0.0
536967                                 0.0                0.0                          0.0                      0.0                              0.0
536983                                 0.0                0.0                          0.0                      0.0                              0.0

In [13]:
# For the desired table, the product amounts are replaced with 1.

df_ger.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  RED SPOT GIFT BAG LARGE  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536527                                   0                  0                            0                        0                                0
536840                                   0                  0                            0                        0                                0
536861                                   0                  0                            0                        0                                0
536967                                   0                  0                            0                        0                                0
536983                                   0                  0                            0                        0                                0

In [14]:
# The above code is valid if product id is not used. If the product id will be used, the function below will work.

def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)


In [15]:
# The new dataframe is composed.

ger_inv_pro_df = create_invoice_product_df(df_ger, id = True)

ger_inv_pro_df.head()

StockCode  10002  10125  10135  11001  15034  15036  15039  16008  16011  16014  16016  16045  16054  16218  16235  16236  16237  16238  16259  17003  20652  20658  20665  20668  20674  20675  20676  20677  20679  20681  20682  20684  20685  20696  20702  20704  20705  20707  20711  20712  20713  20716  20717  20718  20719  20723  20724  20725  20726  20727  20728  20749  20750  20751  20752  20754  20755  20761  20766  20767  20772  20774  20794  20796  20828  20846  20897  20902  20903  20914  20961  20963  20971  20972  20973  20974  20975  20977  20978  20979  20981  20982  20983  20984  20992  20997  20998  21002  21014  21015  21025  21034  21035  21039  21041  21042  21051  21054  21055  21056  21058  21059  21061  21062  21068  21078  21080  21084  21086  21088  21090  21094  21096  21108  21110  21114  21115  21116  21121  21122  21123  21124  21125  21126  21128  21135  21136  21137  21154  21155  21156  21158  21159  21164  21165  21169  21175  21194  21196  21197  21202  21204  21205  21206  21207  21208  21209  21210  21211  21212  21213  21215  21216  21217  21218  21219  21220  21221  21224  21226  21231  21232  21238  21239  21240  21242  21243  21244  21245  21246  21249  21250  21251  21257  21258  21259  21262  21264  21272  21277  21284  21289  21291  21292  21293  21311  21314  21318  21320  21328  21329  21340  21354  21355  21358  21360  21371  21374  21380  21381  21383  21389  21392  21394  21395  21397  21398  21399  21401  21402  21403  21408  21411  21415  21424  21426  21427  21428  21429  21430  21439  21441  21446  21447  21452  21456  21458  21462  21467  21471  21479  21481  21484  21485  21494  21495  21497  21498  21499  21500  21503  21504  21506  21507  21508  21509  21519  21523  21524  21527  21531  21533  21534  21535  21537  21539  21544  21555  21556  21558  21559  21561  21562  21563  21564  21576  21577  21578  21579  21580  21581  21588  21592  21615  21620  21621  21622  21624  21625  21628  21642  21650  21656  21658  21666  21668  21669  21670  21671  21672  21673  21675  21679  21680  21682  21683  21698  21700  21703  21704  21705  21706  21707  21708  21710  21715  21716  21718  21719  21723  21724  21731  21735  21739  21741  21746  21747  21749  21754  21755  21756  21770  21773  21774  21775  21786  21787  21788  21789  21791  21809  21811  21817  21826  21828  21829  21832  21843  21844  21871  21873  21874  21877  21878  21880  21881  21882  21883  21884  21889  21890  21891  21892  21894  21896  21899  21903  21905  21906  21907  21908  21914  21915  21916  21917  21918  21925  21926  21927  21928  21929  21930  21931  21932  21933  21934  21935  21936  21937  21944  21948  21949  21955  21967  21972  21973  21974  21975  21976  21977  21980  21981  21982  21983  21984  21985  21986  21987  21988  21989  21990  21991  21992  21993  22021  22024  22025  22026  22027  22028  22029  22030  22031  22032  22035  22037  22040  22041  22042  22044  22045  22046  22047  22048  22049  22050  22051  22055  22057  22059  22060  22061  22062  22063  22064  22066  22067  22070  22072  22073  22074  22075  22076  22077  22078  22079  22080  22081  22082  22083  22085  22086  22087  22088  22089  22090  22091  22093  22094  22097  22098  22099  22100  22110  22111  22112  22113  22114  22117  22119  22123  22124  22128  22129  22130  22131  22134  22135  22138  22139  22141  22142  22144  22147  22148  22149  22150  22151  22163  22164  22168  22169  22171  22173  22174  22175  22176  22178  22179  22180  22181  22186  22187  22189  22190  22191  22192  22193  22195  22196  22197  22199  22200  22201  22202  22203  22204  22207  22208  22209  22210  22211  22212  22215  22220  22221  22222  22223  22228  22230  22231  22232  22233  22236  22242  22243  22244  22245  22246  22247  22252  22254  22255  22258  22260  22262  22263  22264  22271  22272  22273  22274  22276  22277  22278  22279  22280  22281  22296  22300  22301  22302  22303  22304  22307  22311  22315  22316  22317  22318 

In [16]:
# The below function is used to identify the given id belongs to which item.

def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

In [17]:
product_id = 21987
check_id(df, product_id)

['PACK OF 6 SKULL PAPER CUPS']


In [18]:
# Apriori function calculates the support values of the given dataset. Since a minimum threshold is stated, no lower support values are taken into account. 

# These data are assigned to a new dataset and are sorted according to support values in descending order.

frequent_itemsets = apriori(ger_inv_pro_df, min_support=0.01, use_colnames=True, low_memory=True)

frequent_itemsets.sort_values("support", ascending=False).head()

,support,itemsets
538,0.818381,(POST)
189,0.245077,(22326)
1864,0.225383,"(POST, 22326)"
191,0.157549,(22328)
1931,0.150985,"(22328, POST)"


In [19]:
# Association_rules function calculates confidence, lift and other metric values for the given dataset and the support values. 

rules = association_rules(frequent_itemsets, metric ="support", min_threshold=0.01)

rules.sort_values("support", ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2650,(POST),(22326),0.818381,0.245077,0.225383,0.275401,1.123735,0.024817,1.041850
2651,(22326),(POST),0.245077,0.818381,0.225383,0.919643,1.123735,0.024817,2.260151
2784,(22328),(POST),0.157549,0.818381,0.150985,0.958333,1.171012,0.022049,4.358862
2785,(POST),(22328),0.818381,0.157549,0.150985,0.184492,1.171012,0.022049,1.033038
2414,(22328),(22326),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540


In [20]:
# This function brings the suggested item for the purchased item. 

# The dataset created is sorted according to the lift value. If the product id is in the antecedents column, the suggested item is the first value of the 
# consequents column. This value is added to the recommendation_list. 

def arl_recommender(rules, product_id, rec_count):
    sorted_rules = rules.sort_values("lift", ascending=False)
    recommendation_list = []

    for i, product in sorted_rules["antecedents"].items():
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))

    recommendation_list = list({item for item_list in recommendation_list for item in item_list})

    return recommendation_list[: rec_count]


In [21]:
# With check_id function, we learn what the id of the suggested item belongs to.

check_id(df, arl_recommender(rules, product_id, 1)[0])

['BLUE POLKADOT PLATE ']
